In [ ]:
# new code

In [6]:
'''
Import required packages
'''

from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import StandardScaler
import math
from scipy.stats import pearsonr

import numpy as np
import pandas as pd

from tensorflow import keras
import keras
import keras.backend as K
from keras.models import Sequential
from keras.layers import TimeDistributed, Dense, LSTM, RepeatVector, Dropout, Input, Flatten,Concatenate
from keras.models import load_model
from keras.models import Model
from keras import optimizers
from tensorflow import optimizers

import matplotlib.pyplot as plt
import glob,os
import random
import os,shutil

In [3]:
# data preparation without lookback -- One to One prediction (o2o)
def data_prepare_o2o(dataset,scaler1, scaler2, shuffle):
    
    dataX, dataY = [], []
    scale_feature = scaler1.transform(dataset[:,:-1])
    scale_target = scaler2.transform(dataset[:,-1].reshape(-1, 1))
    for i in range(len(dataset)):
        feature = scale_feature[i]
        dataX.append(feature)
        target = scale_target[i]
        dataY.append(target)
        
    dataX, dataY = np.array(dataX), np.array(dataY)
    number_list = np.arange(0,len(dataX),1)
    random.shuffle(number_list) 
    dataX_new = []
    for index in number_list:
        dataX_new.append(dataX[index])
    dataY_new = []
    for index in number_list:
        dataY_new.append(dataY[index])

    if shuffle:
        return np.array(dataX_new), np.array(dataY_new)
    else:
        return np.array(dataX), np.array(dataY)

In [7]:
'''
Data load
'''

dataset = pd.read_csv(r'C:\Users\Macbook Air\Desktop\NEE_forecast\data\data2_ML_new.csv',index_col = 0)

dynamic_feature_column = dataset.columns[8:18]
static_feature_column1 = dataset.columns[2:8]
static_feature_column2 = dataset.columns[19:]
target = dataset['NEE']

all_sites = dataset['Site'].unique()

result = pd.DataFrame(columns = all_sites, index = ['rmse_train', 'rmse_test', 'corr_train','corr_test'])

In [8]:
site_dataset = dataset.loc[dataset['Site'] == all_sites[0]]

#dynamic_feature = site_dataset[dynamic_feature_column]
#static_feature1 = site_dataset[static_feature_column1]
#static_feature2 = site_dataset[static_feature_column2]

dynamic_dataset = site_dataset.drop(columns=['Unnamed: 0.1', 'Site', 'VegType', 'lat', 'lon', 'timeStamp', 'Year',
       'Month', 'classKoppen', 'SWC', 'soilType', 'high_vegType', 'low_vegType',
       'high_vegCover', 'low_vegCover', 'Elavation'])


# split into train and test sets
train_size = int(len(dynamic_dataset.values) * 0.75)
test_size = len(dynamic_dataset.values) - train_size
train, test = dynamic_dataset.values[0:train_size,:], dynamic_dataset.values[train_size:len(dynamic_dataset.values),:]

# first split train/test and then apply scaler
scaler_x = StandardScaler()
scaler_x.fit(train[:,:-1])

scaler_y = StandardScaler()
scaler_y.fit(train[:,-1].reshape(-1, 1))

# train and test dataset
trainX, trainY = data_prepare_o2o(train, scaler_x, scaler_y, True)
testX, testY = data_prepare_o2o(test, scaler_x, scaler_y, True)

In [10]:
trainY.shape

(42, 1)

In [11]:
### EXP1 ###

from sklearn.preprocessing import StandardScaler
import math
from scipy.stats import pearsonr

for i in range(len(all_sites)):
    site_dataset = dataset.loc[dataset['Site'] == all_sites[i]]

    #dynamic_feature = site_dataset[dynamic_feature_column]
    #static_feature1 = site_dataset[static_feature_column1]
    #static_feature2 = site_dataset[static_feature_column2]

    dynamic_dataset = site_dataset.drop(columns=['Unnamed: 0.1', 'Site', 'VegType', 'lat', 'lon', 'timeStamp', 'Year',
           'Month', 'classKoppen', 'SWC', 'soilType', 'high_vegType', 'low_vegType',
           'high_vegCover', 'low_vegCover', 'Elavation'])


    # split into train and test sets
    train_size = int(len(dynamic_dataset.values) * 0.75)
    test_size = len(dynamic_dataset.values) - train_size
    train, test = dynamic_dataset.values[0:train_size,:], dynamic_dataset.values[train_size:len(dynamic_dataset.values),:]
    
    # first split train/test and then apply scaler
    scaler_x = StandardScaler()
    scaler_x.fit(train[:,:-1])

    scaler_y = StandardScaler()
    scaler_y.fit(train[:,-1].reshape(-1, 1))
    
    # train and test dataset
    trainX, trainY = data_prepare_o2o(train, scaler_x, scaler_y, True)
    testX, testY = data_prepare_o2o(test, scaler_x, scaler_y, True)
    
    '''
    Valina LSTM
    '''
    # define customer optimizer
    #RMSprop= optimizers.RMSprop(lr=0.0001)
    # define special callbacks
    #reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=15, cooldown=30, min_lr=1e-8)
    # define early stop
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0)

    model = Sequential()
    model.add(LSTM(32, input_shape=(trainX.shape[1], 1)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, trainY, epochs=300, validation_split=0.1, batch_size=5, callbacks=[early_stopping], verbose=0)
    
    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    
    trainPredict = scaler_y.inverse_transform(trainPredict)
    trainY = scaler_y.inverse_transform(trainY)
    testPredict = scaler_y.inverse_transform(testPredict)
    testY = scaler_y.inverse_transform(testY)

    # calculate root mean squared error
    trainScore = math.sqrt(mean_squared_error(trainY.squeeze(), trainPredict[:,0]))
    #print('Train Score: %.2f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(testY.squeeze(), testPredict[:,0]))
    #print('Test Score: %.2f RMSE' % (testScore))
    # calculate correlation score
    corr_train, _ = pearsonr(trainY.squeeze(), trainPredict[:,0])
    corr_test, _ = pearsonr(testY.squeeze(), testPredict[:,0])
    
    result[all_sites[i]] = [trainScore, testScore, corr_train, corr_test]
    
    print (all_sites[i])
    

AR-SLu
AT-Neu
AU-ASM
AU-Ade
AU-Cpr
AU-Cum
AU-DaP
AU-DaS
AU-Dry
AU-Emr
AU-Fog
AU-GWW
AU-Gin
AU-How
AU-Lox
AU-RDF
AU-Rig
AU-Rob
AU-Stp
AU-TTE
AU-Tum
AU-Wac
AU-Whr
AU-Wom
AU-Ync
BE-Bra
BE-Lon
BE-Vie
BR-Sa3
CA-Gro
CA-Obs
CA-Qfo
CA-SF1
CA-SF2
CA-SF3
CA-TP1
CA-TP2
CA-TP3
CA-TP4
CA-TPD
CH-Cha
CH-Dav
CH-Fru
CN-Cng
CN-Du2
CN-Du3
CN-HaM
CZ-wet
DE-Geb
DE-Gri
DE-Hai
DE-Kli
DE-Lkb
DE-Lnf
DE-Obe
DE-RuR
DE-RuS
DE-Seh
DE-SfN
DE-Tha
DE-Zrk
DK-Eng
DK-Fou
DK-Sor
ES-Amo
ES-LJu
ES-LgS
FI-Hyy
FI-Jok
FI-Let
FI-Lom
FI-Sod
FR-Gri
FR-LBr
FR-Pue
GH-Ank
GL-ZaF
IT-BCi
IT-CA1
IT-CA2
IT-CA3
IT-Col
IT-Cp2
IT-Cpz
IT-Isp
IT-Lav
IT-MBo
IT-Noe
IT-PT1
IT-Ren
IT-Ro2
IT-SR2
IT-SRo
IT-Tor
MY-PSO
NL-Hor
NL-Loo
RU-Cok
RU-Fyo
RU-Ha1
RU-SkP
RU-Tks
RU-Vrk
SD-Dem
SJ-Adv
SN-Dhr
US-AR1
US-AR2
US-ARM
US-ARb
US-ARc
US-Atq
US-Blo
US-CRT
US-Cop
US-GBT
US-GLE
US-Goo
US-IB2
US-Ivo
US-KS1
US-KS2
US-LWW
US-Lin
US-Los
US-MMS
US-Me1
US-Me2
US-Me4
US-Me5
US-NR1
US-Ne1
US-Ne2
US-Ne3
US-Oho
US-Prr


In [14]:
dynamic_dataset.columns

Index(['ST', 'TD', 'TN', 'P', 'H', 'LE', 'VPD', 'WS', 'SW', 'LW', 'NEE'], dtype='object')

In [12]:
result.to_csv(r'C:\Users\Macbook Air\Desktop\NEE_forecast\result\result_exp1_new.csv')

In [ ]:
# visualization #

In [151]:
'''
Valina LSTM
'''
# define customer optimizer
#RMSprop= optimizers.RMSprop(lr=0.0001)
# define special callbacks
#reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=15, cooldown=30, min_lr=1e-8)
# define early stop
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=2)

model = Sequential()
model.add(LSTM(32, input_shape=(trainX.shape[1], 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=300, validation_split=0.1, batch_size=5, callbacks=[early_stopping], verbose=1)

# model.fit(X, Y, epochs=10, validation_split=0.2, batch_size=5, callbacks=[WandbCallback()])

Epoch 1/300
8/8 [==============================] - 2s 52ms/step - loss: 0.8425 - val_loss: 0.3049
Epoch 2/300
8/8 [==============================] - 0s 8ms/step - loss: 0.7475 - val_loss: 0.3163
Epoch 3/300
8/8 [==============================] - 0s 8ms/step - loss: 0.6654 - val_loss: 0.3436
Epoch 4/300
8/8 [==============================] - 0s 7ms/step - loss: 0.5887 - val_loss: 0.3857
Epoch 5/300
8/8 [==============================] - 0s 7ms/step - loss: 0.5491 - val_loss: 0.4564
Epoch 6/300
8/8 [==============================] - 0s 7ms/step - loss: 0.5172 - val_loss: 0.5291
Epoch 7/300
8/8 [==============================] - 0s 8ms/step - loss: 0.4883 - val_loss: 0.5726
Epoch 8/300
8/8 [==============================] - 0s 7ms/step - loss: 0.4695 - val_loss: 0.5953
Epoch 9/300
8/8 [==============================] - 0s 8ms/step - loss: 0.4482 - val_loss: 0.5685
Epoch 10/300
8/8 [==============================] - 0s 7ms/step - loss: 0.4333 - val_loss: 0.5182
Epoch 11/300
8/8 [==========

8/8 [==============================] - 0s 7ms/step - loss: 0.0899 - val_loss: 0.0427
Epoch 168/300
8/8 [==============================] - 0s 6ms/step - loss: 0.0893 - val_loss: 0.0409
Epoch 169/300
8/8 [==============================] - 0s 6ms/step - loss: 0.0983 - val_loss: 0.0513
Epoch 170/300
8/8 [==============================] - 0s 7ms/step - loss: 0.0891 - val_loss: 0.0426
Epoch 171/300
8/8 [==============================] - 0s 6ms/step - loss: 0.0869 - val_loss: 0.0412
Epoch 172/300
8/8 [==============================] - 0s 7ms/step - loss: 0.0891 - val_loss: 0.0420
Epoch 173/300
8/8 [==============================] - 0s 6ms/step - loss: 0.0871 - val_loss: 0.0433
Epoch 174/300
8/8 [==============================] - 0s 6ms/step - loss: 0.0877 - val_loss: 0.0412
Epoch 175/300
8/8 [==============================] - 0s 6ms/step - loss: 0.0874 - val_loss: 0.0419
Epoch 176/300
8/8 [==============================] - 0s 7ms/step - loss: 0.0869 - val_loss: 0.0411
Epoch 177/300
8/8 [=====

Epoch 250/300
8/8 [==============================] - 0s 7ms/step - loss: 0.0513 - val_loss: 0.0233
Epoch 251/300
8/8 [==============================] - 0s 7ms/step - loss: 0.0520 - val_loss: 0.0244
Epoch 252/300
8/8 [==============================] - 0s 6ms/step - loss: 0.0544 - val_loss: 0.0250
Epoch 253/300
8/8 [==============================] - 0s 6ms/step - loss: 0.0513 - val_loss: 0.0245
Epoch 254/300
8/8 [==============================] - 0s 7ms/step - loss: 0.0519 - val_loss: 0.0276
Epoch 255/300
8/8 [==============================] - 0s 6ms/step - loss: 0.0512 - val_loss: 0.0255
Epoch 256/300
8/8 [==============================] - 0s 7ms/step - loss: 0.0485 - val_loss: 0.0239
Epoch 257/300
8/8 [==============================] - 0s 6ms/step - loss: 0.0508 - val_loss: 0.0251
Epoch 258/300
8/8 [==============================] - 0s 6ms/step - loss: 0.0535 - val_loss: 0.0310
Epoch 259/300
8/8 [==============================] - 0s 6ms/step - loss: 0.0509 - val_loss: 0.0230
Epoch 260/

In [153]:
# invert predictions

import math

trainPredict = scaler_y.inverse_transform(trainPredict)
trainY = scaler_y.inverse_transform([trainY])
testPredict = scaler_y.inverse_transform(testPredict)
testY = scaler_y.inverse_transform([testY])

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY.squeeze(), trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY.squeeze(), testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 0.36 RMSE
Test Score: 0.46 RMSE


In [ ]:
'''

Data pre-proceeing -- ONE TO ONE; dynamic features and target should be scaled

'''
def data_process_NEE(dynamic_feature, target, n_input, n_output, shuffle): 
    
    x1, y= list(),list()
    for i in range(len(dynamic_feature)):
        seq_x1, seq_y = dynamic_feature[], data_2.values[:,0][end_ix1:end_y] # input runoff observation and target runoff 
        seq_x2 = sequence[:,1:][i:end_ix2, :] # input rainfall, tmax and tmin observation
        x1.append(seq_x1)

        y.append(seq_y)
    
    d_input = scaler1.transform(dynamic_feature)
    
    for j in range(len(dataset)): # go through each catchment
        
        
        sequence = scaler.transform(data_1) # introduce MinMax scaler for individual event, only for input data


        for i in range(len(sequence)): # go through each event
            end_ix1 = i + n_input # runoff observation data

            seq_x1, seq_y = sequence[:,0][i:end_ix1], data_2.values[:,0][end_ix1:end_y] # input runoff observation and target runoff 
            seq_x2 = sequence[:,1:][i:end_ix2, :] # input rainfall, tmax and tmin observation
            x1.append(seq_x1)
            
            y.append(seq_y)


    #  only shuffle the sliced in/output sequence when training model
    x1, y = np.array(x1), np.array(y)
    number_list = np.arange(0,len(x1),1)
    random.shuffle(number_list) 
    x1_new = []
    for index in number_list:
        x1_new.append(x1[index])
  
    y_new = []
    for index in number_list:
        y_new.append(y[index])
    
    if shuffle:
        return np.array(x1_new), np.array(y_new)
    else:
        return x1, y


In [ ]:
'''
Simple LSTM structure

'''

######## design LSTM network structure ########
#dim_dense=[512, 256, 256, 128, 64]
# define customer optimizer
RMSprop= optimizers.RMSprop(lr=0.0001)
# define special callbacks
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=15, cooldown=30, min_lr=1e-8)
# define early stop
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=40, verbose=2)

# input of runoff observation, LSTM encoder
input_1 = Input(shape=(train1_x.shape[1],train1_x.shape[2]), name='LSTM1_input') # shape should be 72*1 for runoff observation
LSTM1 = LSTM(256, return_sequences=False)(input_1)
    
#x = TimeDistributed(Dense(2, activation='relu'))(x)
#x = TimeDistributed(Dropout(0.4))(x) # Some dropout for dense layers.
    
main_out = Dense(1, activation='relu')(x) 
#main_out = Flatten()(main_out)

################## model training ####################
model = Model(inputs=[input_1], outputs=main_out)
model.compile(optimizer=RMSprop, loss='mse') 
model.summary()

# fit network 
history = model.fit([train1_x], train_y, epochs=300, batch_size=128,
      validation_data=([val1_x], val_y), callbacks=[reduce_lr,early_stopping], verbose=1)

# save the model
#model.save('/content/drive/My Drive/master_thesis/_code/regional_model/pub_%s.h5' % (num_ite))

In [ ]:
'''
statisc-LSTM Model structure
'''
######## design LSTM network structure ########
dim_dense=[512, 256, 256, 128, 64]
# define customer optimizer
RMSprop= optimizers.RMSprop(lr=0.0001)
# define special callbacks
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=15, cooldown=30, min_lr=1e-8)
# define early stop
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=40, verbose=2)

# input of runoff observation, LSTM encoder
input_1 = Input(shape=(train1_x.shape[1],train1_x.shape[2]), name='LSTM1_input') # shape should be 72*1 for runoff observation
LSTM1 = LSTM(256, return_sequences=False)(input_1)

# input of runoff observation and forecast, LSTM encoder
#input_2 = Input(shape=(train2_x.shape[1],train2_x.shape[2]), name='LSTM2_input') # shape should be (72+24)*n=96*n, for rainfall observation (72) and predictions (24) for n stations (if there is no upstream station, n=1)
#LSTM2 = LSTM(256, return_sequences=False)(input_2)

# input of other non-timeseries data, such as daily or monthly data.
#input_static = Input(shape=(14,), name='static_input') # shape = 14 -- 14 static variables -- 2D in this case with only Input or Dense layer (14,)/3D for LSTM layer (1,14)
#Layer3 = Dense(16, activation='relu')(input_static)

# connect all data
#x = Concatenate()([Layer3, LSTM1, LSTM2]) 
#x = RepeatVector(6)(x)  # 6 hour ahead prediction

# LSTM decoder
#x = LSTM(512, return_sequences=True)(x)

# final fully-connected layer for final result
for dim in dim_dense:
    x = TimeDistributed(Dense(dim, activation='relu'))(x)
    x = TimeDistributed(Dropout(0.4))(x) # Some dropout for dense layers.
    
    
main_out = TimeDistributed(Dense(1, activation='relu'))(x) 
main_out = Flatten()(main_out)

################## model training ####################
model = Model(inputs=[input_static, input_1, input_2], outputs=main_out)
model.compile(optimizer=RMSprop, loss=custom_regional_nse) 
model.summary()

# fit network 
history = model.fit([train_static, train1_x, train2_x], train_y, epochs=300, batch_size=128,
      validation_data=([val_static, val1_x, val2_x], val_y), callbacks=[reduce_lr,early_stopping], verbose=1)

# save the model
model.save('/content/drive/My Drive/master_thesis/_code/regional_model/pub_%s.h5' % (num_ite))



In [ ]:
'''
LSTM Model structure
'''
"""
Read all flood events from each catchment for train and test;
Create paired input and output sequence for model training and testing;
Set up and train LSTM network;
Evaluate model performance on each flood event

"""

for k in range(len(train_index_list)):
    num_ite = k + 1 # count the number of folder
    print (num_ite)

    # read all data
    train_file_all = []
    for i in range(len(train_index_list)):
        catchment = basins[train_index_list[i]]
        train_path = '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/model1/%s_events' % catchment
        trainlist = os.listdir(train_path)
        temp_name = []
        for i in range(len(trainlist)):
          
          temp = train_path + '/'+ trainlist[i]
          
          train_file_all.append(temp)

    test_file_all = []
    for i in range(len(test_index_list)):
        catchment = basins[test_index_list[i]]
        test_path = '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/model1/%s_events' % catchment
        testlist = os.listdir(test_path)
        temp_name = []
        for i in range(len(testlist)):
          
          temp = test_path + '/'+ testlist[i]
          
          test_file_all.append(temp)

    # Create scaler
    df2 = pd.DataFrame()
    for j in range(len(train_file_all)):
        data = pd.read_csv(train_file_all[j], index_col = 0)
        df2 = pd.concat([df2,data],axis=0) 
    temp = df2.values
    min_max_scaler1 = MinMaxScaler() 
    min_max_scaler1.fit(temp) 

    # input for training model
    train1_x, train2_x, train_y, train_static = split_input(pca_score_nor, train_file_all, 24, 6,min_max_scaler1,shuffle = True)
    # input for validating model 
    val1_x, val2_x, val_y, val_static = split_input(pca_score_nor, test_file_all, 24, 6, min_max_scaler1, shuffle = True)
    # reshape train/val data
    train1_x = train1_x.reshape((train1_x.shape[0],train1_x.shape[1],1))
    val1_x = val1_x.reshape((val1_x.shape[0],val1_x.shape[1],1)) 

    ######## design LSTM network structure ########
    dim_dense=[512, 256, 256, 128, 64]
    # define customer optimizer
    RMSprop= optimizers.RMSprop(lr=0.0001)
    # define special callbacks
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=15, cooldown=30, min_lr=1e-8)
    # define early stop
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=40, verbose=2)

    # input of runoff observation, LSTM encoder
    input_1 = Input(shape=(train1_x.shape[1],train1_x.shape[2]), name='LSTM1_input') # shape should be 72*1 for runoff observation
    LSTM1 = LSTM(256, return_sequences=False)(input_1)

    # input of runoff observation and forecast, LSTM encoder
    input_2 = Input(shape=(train2_x.shape[1],train2_x.shape[2]), name='LSTM2_input') # shape should be (72+24)*n=96*n, for rainfall observation (72) and predictions (24) for n stations (if there is no upstream station, n=1)
    LSTM2 = LSTM(256, return_sequences=False)(input_2)

    # input of other non-timeseries data, such as daily or monthly data.
    input_static = Input(shape=(14,), name='static_input') # shape = 14 -- 14 static variables -- 2D in this case with only Input or Dense layer (14,)/3D for LSTM layer (1,14)
    Layer3 = Dense(16, activation='relu')(input_static)

    # connect all data
    x = Concatenate()([Layer3, LSTM1, LSTM2]) 
    x = RepeatVector(6)(x)  # 6 hour ahead prediction

    # LSTM decoder
    x = LSTM(512, return_sequences=True)(x)

    # final fully-connected layer for final result
    for dim in dim_dense:
      x = TimeDistributed(Dense(dim, activation='relu'))(x)
      x = TimeDistributed(Dropout(0.4))(x) # Some dropout for dense layers.
    main_out = TimeDistributed(Dense(1, activation='relu'))(x) 
    main_out = Flatten()(main_out)

    ################## model training ####################
    model = Model(inputs=[input_static, input_1, input_2], outputs=main_out)
    model.compile(optimizer=RMSprop, loss=custom_regional_nse) 
    model.summary()

    # fit network 
    history = model.fit([train_static, train1_x, train2_x], train_y, epochs=300, batch_size=128,
              validation_data=([val_static, val1_x, val2_x], val_y), callbacks=[reduce_lr,early_stopping], verbose=1)
    
    # save the model
    model.save('/content/drive/My Drive/master_thesis/_code/regional_model/pub_%s.h5' % (num_ite)) 

    ##################  Evaluate model performance on each event ##################

    model_pub = load_model('/content/drive/My Drive/master_thesis/_code/regional_model/pub_%s.h5' % (num_ite), 
                           custom_objects={'custom_regional_nse': custom_regional_nse})
    catchment = basins[test_index_list]
    basins = nse_table.columns
    
    test_file_all = []
    for i in range(len(test_index_list)): # test on test basins
        catchment = basins[test_index_list[i]]
        test_path = '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/model1/%s_events' % catchment
        testlist = os.listdir(test_path)
        dataset = testlist
        ed_nse = []
        ed_hour = []
        ed_volume = []
        ed_peak = []
        dates = []

        # pre-processing the input sequence for performance evaluation -- event-wise
        for j in range(len(dataset)):
            x1, x2,y, seq_static = list(),list(), list(), list()
            data_1 = pd.read_csv(test_path+'/'+dataset[j], index_col = 0)
            data_2 = pd.read_csv(test_path+'/'+dataset[j], index_col = 0) # not normalize y(target)
            date = dataset[j][:10]
            # MinMax for individual event
            n_input = 24
            n_output = 6
            sequence = min_max_scaler1.transform(data_1) # same scaler as training dataset

            # add static variable
            temp_static = np.zeros((1,14))
            temp_static = pca_score_nor[catchment].values # each basin one static input only one step
            for i in range(len(sequence)):
                # find the end of this pattern
                end_ix1 = i*n_output + n_input # runoff
                end_ix2 = i*n_output + n_input + n_output # others
                end_y = end_ix1 + n_output
                # check if we are beyond the sequence
                if end_ix1 > len(sequence):
                    break
                if end_ix2 > len(sequence):
                    break
                if end_y > len(sequence):
                    break
                seq_x2 = sequence[:,1:][i*n_output:end_ix2, :] 
                seq_x1, seq_y = sequence[:,0][i*n_output:end_ix1], data_2.values[:,0][end_ix1:end_y] 
                x1.append(seq_x1)
                x2.append(seq_x2)
                y.append(seq_y)
                seq_static.append(list(temp_static))

            #event-wise model performance evaluation
            x1, x2, y, seq_static = np.array(x1), np.array(x2), np.array(y), np.array(seq_static)
            predictions = []
            x1 = x1.reshape((x1.shape[0],x1.shape[1],1))

            predict = model_pub.predict([seq_static, x1,x2]) 
            predictions.append(predict)
            dates.append(date) 
            ed_peak.append(np.max(np.array(predictions).flatten()))
            ed_nse.append(r2_score(np.array(y).flatten(), np.array(predictions).flatten()))

            # peak hour
            hour_pre = np.argmax(np.array(predictions).flatten())
            hour_obs = np.argmax(np.array(y).flatten())
            ed_hour.append((hour_pre - hour_obs))
            
            # peak discharge
            max_pre = np.max(np.array(predictions).flatten())
            max_obs = np.max(np.array(y).flatten())
            ed_volume.append((max_pre - max_obs)/max_obs)

        temp = np.array([dates, ed_peak, ed_hour, ed_volume]) # statistics for event-wise prediction at each catchment
        output = pd.DataFrame([temp[0,:],temp[1,:],temp[2,:],temp[3,:]], index=['date', 'flow', 'hour','ratio'])
        output.to_csv('/content/drive/My Drive/master_thesis/_code/regional_model/peak_pub/%s_peak_pub.csv' % (catchment))
        nse_table[catchment] = np.median(ed_nse) # collect NSE score for each basin in one test fold


